In [16]:
import sys 
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Clearvale/jake_scripts/Amy_flywheel_scripts/')

import numpy as np
import pandas as pd
import os

In [17]:
z_data = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/ClearMem_Z_Average.csv')
z_data = z_data[['SubID', 'z_ave', 'PSWQ_total', 'WBSI_total', 'RRS_total', 'RRS_depression', 'RRS_brooding', 'RRS_reflection']]
z_data = z_data.dropna()

from scipy.stats import zscore
z_data['br_z_ave'] = z_data['z_ave']
z_data.drop('z_ave', axis=1, inplace=True)
z_data['b_z_ave'] = (zscore(z_data['PSWQ_total']) + zscore(z_data['WBSI_total']) + zscore(z_data['RRS_brooding']))/3
z_data['brd_z_ave'] = (zscore(z_data['PSWQ_total']) + zscore(z_data['WBSI_total']) + zscore(z_data['RRS_brooding']) + zscore(z_data['RRS_reflection']) + zscore(z_data['RRS_depression']))/5

In [18]:
#z_data

In [23]:
subj_accuracy = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/predictions/classifiers/accuracy.csv').iloc[:, 1:7]
subj_evidence = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/predictions/classifiers/evidence.csv').iloc[:, 1:6]

In [24]:
subj_accuracy.subject_id = subj_accuracy.subject_id.str.replace('clearmem_v1_sub', '').astype(int)
subj_evidence.subject_id = subj_evidence.subject_id.str.replace('clearmem_v1_sub', '').astype(int)

subj_accuracy.columns = ['SubID', 'maintain_acc', 'replace_acc', 'suppress_acc', 'clear_acc', 'total_acc']
subj_evidence.columns = ['SubID', 'maintain_ev', 'replace_ev', 'suppress_ev', 'clear_ev']

acc_ev = pd.merge(subj_accuracy, subj_evidence, on='SubID')

In [26]:
acc_ev.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/dispersion_data/derosa_task_classifiers.csv')

In [27]:
acc_ev_z = pd.merge(acc_ev, z_data, on='SubID')
#acc_ev_z

In [28]:
def run_regression(data, target, y_vars, interaction=None):
    import pandas as pd
    import statsmodels.api as sm
    import statsmodels.formula.api as smf

    if interaction is not None:
        # Creating the formula with interaction terms if there are multiple y_vars
        if len(y_vars) > 1:
            joined_vars = ' * '.join(y_vars)
        else:
            joined_vars = y_vars[0]
        
        new_y_vars = [joined_vars]
        formula = f'{target[0]} ~ {joined_vars}'

    else:
        joined_vars = ' + '.join(y_vars)
        new_y_vars = [joined_vars]
        formula = f'{target[0]} ~ {joined_vars}'

    # Fit the regression model using the formula
    model = smf.ols(formula=formula, data=data).fit()

    # Print the full regression output
    summary = model.summary()

    var = pd.DataFrame(summary.tables[0].data).iloc[0, 1]

    table1 = pd.DataFrame(summary.tables[0].data).iloc[:4, 3:].T.assign(var=var)
    table1.columns = ['r2', 'adjr2', 'fstat', 'pval', 'var']
    table1 = table1[['var', 'r2', 'adjr2', 'fstat', 'pval']]

    table2_cols = pd.DataFrame(summary.tables[1].data).loc[0].to_list() + ['var']
    table2_cols[0] = 'parameter'
    table2 = pd.DataFrame(summary.tables[1].data).iloc[1:].assign(var=var)
    table2.columns = table2_cols
    table2 = table2[['var', 'parameter', 'coef', 'std err', 't', 'P>|t|']]

    df = pd.merge(table1, table2, how='outer', left_on='var', right_on='parameter')

    # Combine 'var_x' and 'var_y' into a new column 'var'
    df['var'] = df['var_x'].fillna(df['var_y'])

    # Drop the original 'var_x' and 'var_y' columns
    df.drop(columns=['var_x', 'var_y'], inplace=True)

    # Reorder columns to place 'var' at the front
    cols = ['var'] + [col for col in df.columns if col != 'var']
    df = df[cols]
    df = df.assign(formula=formula)

    return df


def regression_function(data, variable_list, interaction):

    import itertools

    # Given list
    variables = variable_list
    targets = [['PSWQ_total'], ['brd_z_ave']]

    # Create a list to hold all combinations
    all_combinations = []

    # Generate combinations for each length from 1 to the length of the list
    for r in range(1, len(variables) + 1):
        combinations = list(itertools.combinations(variables, r))
        all_combinations.extend(combinations)

    # Convert each tuple to a list
    list_combinations = [list(item) for item in all_combinations]

    regression_combinations = []
    for i in targets:
        for j in list_combinations:
            regression_combinations.append(run_regression(data, i, j, interaction))

    output_regressions = pd.concat(regression_combinations)

    # List of columns to convert
    columns_to_convert = ['r2', 'adjr2', 'fstat', 'pval', 'coef', 'std err', 't', 'P>|t|']

    # Convert each specified column to numeric, handling non-numeric values by converting them to NaN
    for column in columns_to_convert:
        output_regressions[column] = pd.to_numeric(output_regressions[column], errors='coerce')

    return output_regressions

In [29]:
acc_mean_list = ['maintain_acc', 'replace_acc', 'suppress_acc', 'clear_acc', 'total_acc']
acc_mean_reg = regression_function(acc_ev_z, acc_mean_list, interaction=None)

In [30]:
(acc_mean_reg.query('pval < .05')
 .query('var == "brd_z_ave"')
 .sort_values('fstat', ascending=False)
 .formula.to_list()
)

['brd_z_ave ~ clear_acc',
 'brd_z_ave ~ suppress_acc',
 'brd_z_ave ~ total_acc',
 'brd_z_ave ~ maintain_acc',
 'brd_z_ave ~ replace_acc + total_acc']

In [31]:
ev_mean_list = ['maintain_ev', 'replace_ev', 'suppress_ev', 'clear_ev']
ev_mean_reg = regression_function(acc_ev_z, ev_mean_list, interaction=None)

In [32]:
(ev_mean_reg.query('pval < .05')
 .query('var == "brd_z_ave"')
 .sort_values('fstat', ascending=False)
 .formula.to_list()
)

['brd_z_ave ~ maintain_ev', 'brd_z_ave ~ clear_ev', 'brd_z_ave ~ suppress_ev']

In [13]:
#acc_ev_mean_list = ['maintain_acc', 'replace_acc', 'suppress_acc', 'clear_acc', 'total_acc', 'maintain_ev', 'replace_ev', 'suppress_ev', 'clear_ev']
#acc_ev_mean_reg = regression_function(acc_ev_z, acc_ev_mean_list, interaction=None)

In [14]:
#(acc_ev_mean_reg.query('pval < .05')
# .query('var == "brd_z_ave"')
# .sort_values('fstat', ascending=False)
 #.formula.to_list()
#)

In [11]:
sub_disp_ecc_means = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/sub_disp_ecc_means_z.csv')


sub_disp_ecc_acc_ev_means = pd.merge(sub_disp_ecc_means, acc_ev, on='SubID')

In [19]:
sub_disp_ecc_acc_ev_means.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/sub_disp_ecc_acc_ev_means.csv', index=False)

In [ ]:
all_list = [ 'vn_fpcn_mean', 'vn_dmn_mean', 
'fpcn_dmn_mean', 'vn_mean', 'fpcn_mean',
'dmn_mean', 'overall_mean', 'main_mean', 'replace_mean',
'suppress_mean', 'clear_mean', 'vn_mean_ecc',
'fpcn_mean_ecc', 'dmn_mean_ecc', 'overall_mean_ecc', 'maintain_acc',
'replace_acc', 'suppress_acc', 'clear_acc', 'total_acc', 'maintain_ev',
'replace_ev', 'suppress_ev', 'clear_ev']

all_mean_reg = regression_function(sub_disp_ecc_acc_ev_means, all_list, interaction=None)

In [ ]:
cor_matt = sub_disp_ecc_acc_ev_means[['dmn_mean', 'replace_mean', 'vn_mean_ecc', 'dmn_mean_ecc', 'clear_acc', 'maintain_ev', 'fpcn_dmn_mean']].corr(method='spearman')

import seaborn as sns
sns.clustermap(cor_matt, center=0, cmap='bwr', annot=True)


In [ ]:
(all_mean_reg.query('pval < .05')
 .query('var == "brd_z_ave"')
 .sort_values('pval', ascending=True)
 #.formula.to_list()
)

In [26]:
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler

# Assuming sub_disp_ecc_acc_ev_means is your DataFrame
X = sub_disp_ecc_acc_ev_means.drop(['SubID', 'PSWQ_total', 'WBSI_total', 'RRS_total', 
                                    'RRS_depression', 'RRS_brooding', 'RRS_reflection', 
                                    'br_z_ave', 'b_z_ave', 'brd_z_ave'], axis=1)
y = sub_disp_ecc_acc_ev_means['brd_z_ave'].values.ravel()  # Using .ravel() to create a 1D array

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Setting up cross-validation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Define the LassoCV model
model = LassoCV(cv=cv, max_iter=10000)  # Removed normalize parameter

# Fit model
model.fit(X_scaled, y)

# Check coefficients
coef = model.coef_

# Identifying non-zero coefficients (selected features)
selected_features = np.where(coef != 0)[0]

print("Selected features indices:", selected_features)
print("Coefficients of selected features:", coef[selected_features])

# Mapping selected indices to feature names based on remaining columns in X
feature_names = X.columns[selected_features]
print("Names of selected features:", feature_names)


Selected features indices: [ 9 12 15 18 23 25]
Coefficients of selected features: [-0.07928836 -0.00832382  0.09785993 -0.06844785 -0.0304262  -0.07307126]
Names of selected features: Index(['dmn_mean', 'replace_mean', 'vn_mean_ecc', 'dmn_mean_ecc', 'global_acc',
       'maintain_ev'],
      dtype='object')


In [83]:
import pandas as pd
import numpy as np

def bootstrap_regression(data, columns, n_iterations=100):
    # Store results in a list of DataFrames
    
    global regression_function
    
    results = []
    # Perform bootstrapping
    for i in range(n_iterations):
        
        # Sample with replacement
        sample = data.sample(n=len(data), replace=True)

        # Run the regression function
        result = regression_function(sample, columns, interaction=None)
        print
        # Append the result to the list
        results.append(result)
    
    # Concatenate all results into a single DataFrame
    final_results = pd.concat(results, ignore_index=True)
    
    return final_results


# Assuming sub_disp_ecc_acc_ev_means is your DataFrame and all_list is defined as before
all_list = ['dmn_mean', 'replace_mean', 'vn_mean_ecc', 'dmn_mean_ecc', 'clear_acc', 'maintain_ev', 'fpcn_dmn_mean']

# Call the bootstrap function with your dataset, regression function, and the list of all features
bootstrapped_results = bootstrap_regression(sub_disp_ecc_acc_ev_means, all_list, n_iterations=100)

In [84]:
bootstrapped_results.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/operation_regressions/bootstrap_regressions/boot_results.csv')

In [32]:
#all_list = [ 'vn_fpcn_mean', 'vn_dmn_mean', 
#'fpcn_dmn_mean', 'vn_mean', 'fpcn_mean',
#'dmn_mean', 'overall_mean', 'main_mean', 'replace_mean',
#'suppress_mean', 'clear_mean', 'vn_mean_ecc',
#'fpcn_mean_ecc', 'dmn_mean_ecc', 'overall_mean_ecc', 'maintain_acc',
#'replace_acc', 'suppress_acc', 'clear_acc', 'total_acc', 'maintain_ev',
#'replace_ev', 'suppress_ev', 'clear_ev']

In [82]:
import pandas as pd
import seaborn as sns
# Assuming bootstrapped_results is already loaded as a DataFrame
# First, let's group by 'formula' and calculate the mean and standard deviation of 'pval'
# Assuming 'fstat' and 'adjr2' are columns in bootstrapped_results DataFrame
# Group by 'formula' and calculate the mean and standard deviation of 'pval', and mean of 'fstat' and 'adjr2'
stats = bootstrapped_results.groupby('formula').agg({
    'pval': ['mean', 'std'],
    'fstat': ['mean', 'std'],
    'adjr2': ['mean', 'std']
})

# Rename columns for clarity
stats.columns = ['mean_pval', 'std_pval', 'mean_fstat', 'std_fstat', 'mean_adjr2', 'std_adjr2']

# Sort the formulas first by mean_pval in ascending order (lower means better),
# and then by std_pval in ascending order (lower variability is better)
sorted_stats = stats.sort_values(by=['mean_pval', 'std_pval'])

# Now, select the top 10 formulas
top_10_formulas = sorted_stats.head(10).reset_index()

top_10_formulas.formula = top_10_formulas.formula.str.replace('brd_z_ave ~', '')
top_10_formulas.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/operation_regressions/bootstrap_regressions/top_10_formulas.csv')

